In [1]:
import cobra
import time
import numpy as np
import more_itertools as mi
import multiprocessing as mp

In [2]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [3]:
from cobra import Model, Reaction, Metabolite

In [4]:
start=time.time()

In [5]:
cobra.Configuration().solver="cplex"

In [6]:
%%time
model=cobra.io.read_sbml_model("./iNovo_base_2022.xml")

CPU times: user 710 ms, sys: 13.2 ms, total: 723 ms
Wall time: 734 ms


In [9]:
sol=model.optimize()

In [7]:
model2=model.copy()

In [12]:
r=model2.reactions.get_by_id("EX_"+lista[1])
r

Reaction identifier,EX_exVA
Name,vanillic acid exchange
Memory address,0x7fe654fc1dd8
Stoichiometry,exVA <-- vanillic acid <--
GPR,
Lower bound,-1000.0
Upper bound,0.0


In [13]:
lista=['exC00031', 'exVA','expHBA' , 'exSA', 'exS',  'exPCA', 'exV', 'exFA', 'exGDK', 'exSDK', 'exSSGGE', 'exSRGGE', 'exRSGGE', 'exRRGGE']

In [24]:
for i in range(2,len(lista)):
    r=model2.reactions.get_by_id("EX_"+lista[i])
    r.lower_bound=0
r=model2.reactions.get_by_id("EX_"+lista[0])
r.lower_bound=-2
r=model2.reactions.get_by_id("EX_"+lista[1])
r.lower_bound=0

In [15]:
reaction = Reaction('ExportaPDC')
reaction.compartment = 'e'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    model2.metabolites.exPDC: -1.0,
})
model2.add_reactions([reaction])

In [16]:
for j in range(len(model2.reactions)):
    r=model2.reactions[j]
    if r.id=="biomass":
        biomass=j
    if r.id=="ExportaPDC":
        objetivo=j

In [25]:
model2.objective=model2.reactions[biomass]
model2.objective_direction="max"
model2.slim_optimize()

0.24374020525258303

In [18]:
nombres_genes=["SARO_RS14300","Saro_2864","SARO_RS14530"]

In [21]:
with model2:
    model2.reactions[biomass].lower_bound=0.1
    for nombre in nombres_genes:
        model2.genes.get_by_id(nombre).knock_out()
    model2.objective=model2.reactions[objetivo]
    sol=model2.optimize(objective_sense="minimize")
    print(sol)

<Solution 0.000 at 0x7fe654be3710>


In [13]:
for g in nombres_genes:
    print(g,len(model2.genes.get_by_id(g).reactions))

SARO_RS14300 2
Saro_2864 1
SARO_RS14530 2


In [14]:
def creaSoportes(modelo,numSoportes,reacciones,valores):
    soportes=set()
    while len(soportes)<numSoportes:
        with modelo:
            for k in range(len(valores)):
                r=modelo.reactions[reacciones[k]]
                r.bounds=valores[k]
                    
            r2=np.random.choice(modelo.reactions)
            modelo.objective=r2
            
            sol=modelo.optimize(objective_sense="maximize")

            if sol.status=="optimal":
                sop=frozenset([j for j in range(len(sol.fluxes)) if abs(sol.fluxes[j])>10**-12])
                soportes.add(sop)
        
    return(soportes)

def identificaTipo(nombre):
    if "(" in nombre:
        indice1=nombre.index("(")
        nombre2=nombre[indice1+1:]
        indice2=nombre2.index(")")
        nombre3=nombre2[:indice2] 
        if " and " in nombre3:
            tipo="SOP"
        if " or " in nombre3:
            tipo="POS"
    else:
        tipo="SOP"
    return(tipo)

def toPOS(nombre,tope=3):

    productos=set()    
    tipo=identificaTipo(nombre)

    if tipo=="SOP":
        sumandos=nombre.split(" or ")
        for sumando in sumandos:
            if " and " in sumando:
                casos=sumando.split(" and ")
                miCaso=set()
                for caso in casos:
                    caso2=caso.replace("(","")
                    caso3=caso2.replace(")","")
                    caso4=caso3.replace(" ","")
                    miCaso.add(caso4)

                productos.add(frozenset(miCaso))
            else:
                productos.add(frozenset([sumando]))

        CS=set([frozenset()])
        for producto in productos:
            CS2=set()
            for C in CS:
                if len(set(C).intersection(producto))==0:
                    for a in producto:
                        C2=set(C.copy())
                        C2.add(a)
                        if len(C2)<=tope:
                            CS2.add(frozenset(C2))
                else:
                    CS2.add(C)
            CS=CS2

        CS3=set()
        for C in CS:
            bien=True
            for C2 in CS:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    else:
        CS2=set()
        productos = nombre.split(" and ")
        for producto in productos:
            CS=set()
            componentes=producto.split(" or ")
            for C in componentes:
                C2=C.replace("(","")
                C3=C2.replace(")","")
                C4=C3.replace(" ","")
                CS.add(C4)
            CS2.add(frozenset(CS))
        CS3=set()
        for C in CS2:
            bien=True
            for C2 in CS2:
                if not C==C2 and C2.issubset(C):
                    bien=False
                    break
            if bien:
                CS3.add(C)
    return(CS3)

def asociar(modelo,tope=4):
    verbose=False
    total=0
    asociados=dict()
    reacciones=dict()

    for num in range(len(modelo.reactions)):
        nombre=modelo.reactions[num].gene_reaction_rule
        CS3=toPOS(nombre,tope)
        k=0
        while k<=tope:
            mas=0
            nivel=set([C for C in CS3 if len(C)==k and not C==frozenset({''})])
            if len(nivel)>0:
                mios=set()
                for caso in nivel:
                    mio=set()
                    for a in caso:
                        mio.add(a)
                    mios.add(frozenset(mio))

                asociados[(modelo.reactions[num],k)]=mios

                for caso in mios:
                    if not caso in reacciones:
                        reacciones[caso]=set([modelo.reactions[num]])
                    else:
                        reacciones[caso].add(modelo.reactions[num])
                mas+=len(nivel)
            total+=mas
            k+=1    
        if verbose:
            print(num,len(modelo.reactions[num].genes),mas)

    return(asociados,reacciones)
    
    
def soporteG(soporte,tope,modelo=model2):
    sopG=set()
    for j in soporte:
        r=modelo.reactions[j]
        for k in range(1,tope+1):
            if (r,k) in asociados:
                sopG.update(asociados[(r,k)])

    sopG2=set()
    for caso in sopG:
        poner=True
        for caso2 in sopG:
            if not caso==caso2 and caso2.issubset(caso):
                poner=False
                break
        if poner:
            sopG2.add(caso)
    return(sopG2)

def chequea(inicio,fin):
    respuestas=set()
    for i in range(inicio,fin):
        if i<len(finales):
            C=finales[i]
            vale=True
            if C in powers:
                po=powers[C]
            else:
                po=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                powers[C]=po

            if len(po.intersection(todosMCS))>0 or len(evitar.intersection(C))>0:
                vale=False

            if vale:
                for sop in misSoportes:
                    if len(po.intersection(sopG[sop]))==0:           
                        vale=False
                        break
            if vale:
                respuestas.add(C)

    return(respuestas)


In [15]:
def CSV2(modelo,tope,misReacciones,misValores,verbose,evita):
    global tiempos
    global todosMCS
    global evitar
    global misSoportes
    global listaMias
    global listaFinales
    global sopG
    
    global finales
    
    listaMias=[]
    salto=6
    
    evitar=evita
    listaMCS=[]
    listaMCS.append(set())
    
    listaCandidatos=[]
    listaCandidatos.append(set([frozenset()]))
    todosMCS=set()
    cutsets=[set()]
    candidatos=[set()]

    ts=0
    
    for i in range(1,tope+1):
        if i==1:            
            genes=set()
            cont=0
            for sop in misSoportes:
                if not sop in sopG:
                    sopG[sop]=soporteG(sop,tope,modelo)
                nuevos=[caso for caso in sopG[sop] if len(caso)==1 and len(evitar.intersection(caso))==0]
                if cont==0:
                    genes.update(nuevos)
                else:
                    genes=genes.intersection(nuevos)
                cont+=1
            if verbose:
                print(len(genes))
            descartados=set()
            CST=set()
            for caso in genes:
                if not caso in descartados:
                    with modelo:
                        for g in caso:
                            modelo.genes.get_by_id(g).knock_out()
                        for k in range(len(misValores)):
                            r=modelo.reactions[misReacciones[k]]
                            r.bounds=misValores[k]
                        modelo.objective=0
                        sol=modelo.slim_optimize(error_value=-1)
                        if sol==-1:
                            CST.add(caso)
                            todosMCS.add(caso)
                        else:
                            sol2=modelo.optimize()
                            sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                            if not sop in sopG:
                                sopG[sop]=soporteG(sop,tope,modelo)
                                misSoportes.add(sop)
                            nuevos=[caso for caso in sopG[sop] if len(caso)==1]
                            descartados.update(genes.difference(nuevos))
            
            cutsets.append(CST)
            candidatos.append(genes.difference(CST))
            
            print("Final Fase 1 con",len(candidatos[1]),"candidatos y",len(cutsets[1]),"cutsets")

            print()
       
        ############################## Final Fase 1 ############################################
    
        else:
            print("Inicio de Fase",i,"con",len(misSoportes),"soportes")
            iMio=i
            
            ############################## Parte 1.- Berge #################################### 
            st1=time.time()
            CST2=set()
            lista=set([frozenset()])
            descartados=set()
            cont2=0
            long=dict()
            for sop in misSoportes:
                long[sop]=len(sopG[sop])
            sortedSupports= sorted(long.items(), key=lambda x:x[1])
            finales=set()
            
            for t in range(len(misSoportes)):
                ################ Para cada soporte #################
                item=sortedSupports[t]
                sop=item[0]
                lista2=set()
                for caso in lista:
                    sst=time.time()
                    if caso in powers:
                        po=powers[caso]
                    else:
                        po=set([frozenset(a) for a in mi.powerset(caso) if len(a)>0])
                        powers[caso]=po
                    esta=(len(po.intersection(sopG[sop]))>0)
                    ts+=(time.time()-sst)
                    if esta:
                        lista2.add(caso)
                    else:
                        for caso2 in sopG[sop]:
                            if len(caso.union(caso2))<iMio:
                                nuevo=caso.union(caso2)
                                lista2.add(nuevo)
                            if len(caso.union(caso2))==iMio:
                                nuevo=caso.union(caso2)
                                finales.add(nuevo)
                        
                
                lista4=set()
                for C in lista2:
                    sst=time.time()
                    if C in powers:
                        po2=powers[C]
                    else:
                        po2=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                        powers[C]=po2
                    if len(po2.intersection(todosMCS))==0 and len(evitar.intersection(C))==0:
                        lista4.add(C)
                    ts+=(time.time()-sst)
                cont2+=1
                lista=lista4
                lista3=[C for C in lista if len(C)<i]
                if verbose:
                    print("Mal",len(lista3))
                if len(lista)>0 and len(lista3)==0:
                    break
                
            
                    #break
            ########################## He terminado de procesar los soportes ###############
            
            if verbose:
                print("Parte paralela")
            
            lista3=[C for C in lista if len(C)<i]
            
            
            
            misFinales=list(finales)
            if verbose:
                print("Total",len(misFinales))
            lista=set()
            numLista=10**6
            r=0
            while r*numLista<len(misFinales):
                maximo=(r+1)*10**6
                if len(misFinales)<maximo:
                    maximo=len(misFinales)
                finales=misFinales[r*numLista:maximo]
                if verbose:
                    print(r,"Finales",r*numLista,maximo)
                r+=1
                

                finales=list(finales)


                cpus=mp.cpu_count()
                cpus=8

                listas=[]

                k=0
                argus=[]
                num=int(np.floor(len(finales)/(cpus-1)))
                while k*num<len(finales):
                    argus.append((k*num,(k+1)*num))
                    k+=1

                with mp.Pool(processes=cpus) as mp_pool:
                    results=mp_pool.starmap(chequea,argus)

                for result in results:
                    lista.update(result)



            st1=time.time()-st1
            st2=time.time()
            
            cont2=1
            
            lista2=set()
            
            listaL=list(lista)
            if verbose:
                print(len(lista))
            
            i2=0
            while i2 < len(listaL):
                if verbose:
                    print(i2,len(listaL),len(listaL)-len(descartados))
                casos=[]
                
                contCasos=0
                for k in range(i2,len(listaL)):
                    if not listaL[k] in descartados:
                        casos.append(listaL[k])
                        contCasos+=1
                    if contCasos>=salto:
                        break
                i2=k+1
                
                cont3=0
                if len(casos) > 0:
                    
                    with modelo:
                        for caso in casos:
                            descartados.add(caso)
                            for g in caso:
                                modelo.genes.get_by_id(g).knock_out()
                        
                        for k in range(len(misValores)):
                            r=modelo.reactions[misReacciones[k]]
                            r.bounds=misValores[k]
                        
                        sol=modelo.slim_optimize(error_value=-1)
                        if sol==-1:
                            for caso in casos:
                                lista2.add(caso)
                        else:
                            sol2=modelo.optimize()
                            sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                            if not sop in sopG:
                                sopG[sop]=soporteG(sop,tope,modelo)
                                misSoportes.add(sop)
                            nuevos=[caso for caso in sopG[sop] if len(caso)>0]
                            for C in lista.difference(descartados): 
                                sst=time.time()
                                if C in powers:
                                    sub=powers[C]
                                else:
                                    sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                                    powers[C]=sub
                                if len(sub.intersection(sopG[sop]))==0:
                                    descartados.add(C)
                                ts+=(time.time()-sst)
                if verbose:
                    print(cont2,i2,contCasos,len(lista.difference(descartados)))
                cont2+=1
            if verbose:
                print()
                print("Lista2",len(lista2))
                print()
            
            cutsetsT=set()
            
            st2=time.time()-st2
            st3=time.time()
            
            if verbose:
                print("Sin salto")
            descartados=set()
            cont=0
            for caso in lista2:
                if verbose:
                    print(len(lista2),len(lista2.difference(descartados)),len(cutsetsT))
                cont+=1
                if not caso in descartados:
                    descartados.add(caso)
                    with modelo:
                        for g in caso:
                            modelo.genes.get_by_id(g).knock_out()
                        for k in range(len(misValores)):
                            r=modelo.reactions[misReacciones[k]]
                            r.bounds=misValores[k]

                        sol=modelo.slim_optimize(error_value=-1)
                        if sol==-1:
                            cutsetsT.add(caso)
                            todosMCS.add(caso)
                        else:
                            sol2=modelo.optimize()
                            sop=frozenset([j for j in range(len(modelo.reactions)) if abs(sol2.fluxes[j])>10**-8])
                            if not sop in sopG:
                                sopG[sop]=soporteG(sop,tope,modelo)
                                misSoportes.add(sop)
                            nuevos=[caso for caso in sopG[sop] if len(caso)>0]
                            for C in lista2.difference(descartados): 
                                sst=time.time()
                                sub=set([frozenset(a) for a in mi.powerset(C) if len(a)>0])
                                if len(sub.intersection(sopG[sop]))==0:
                                    descartados.add(C)
                                ts+=(time.time()-sst)
                    if verbose:
                        print(cont,len(lista2.difference(descartados)),len(cutsetsT))     
                
            st3=time.time()-st3

            cutsets.append(cutsetsT)
        if i>1:
            tiempos.append([st1,st2,st3])
    
    #print(ts)
    return(cutsets)
                        

In [85]:
misReacciones, misValores = [biomass,objetivo],[[0.1,1000],[0,0]]

In [86]:
%%time
tope=4
asociados,reacciones=asociar(model2,tope)
sopG=dict()
misSoportes=creaSoportes(model2,tope,misReacciones,misValores)
for sop in misSoportes:
    sopG[sop]=soporteG(sop,tope,model2)

CPU times: user 93.8 ms, sys: 1.06 ms, total: 94.9 ms
Wall time: 94.3 ms


In [87]:
%%time
tiempos=[]
powers=dict()
CS1=CSV2(model2,tope,misReacciones,misValores,False,set())

Final Fase 1 con 12 candidatos y 168 cutsets

Inicio de Fase 2 con 13 soportes
Inicio de Fase 3 con 51 soportes
Inicio de Fase 4 con 201 soportes
CPU times: user 17.3 s, sys: 543 ms, total: 17.9 s
Wall time: 20.8 s


In [88]:
for i,C in enumerate(CS1):
    print(i,len(C))

0 0
1 168
2 72
3 38
4 91


Comprobar que todos son cutsets

In [89]:
%%time

for i in range(1,len(CS1)):
    print(i)
    for caso in CS1[i]:
        with model2:
            for g in caso:
                model2.genes.get_by_id(g).knock_out()
            model2.reactions[biomass].bounds=[1,1000]
            model2.reactions.ExportaPDC.lower_bound=0
            model2.reactions.ExportaPDC.upper_bound=0

            sol=model2.slim_optimize(error_value=-1)
            if not sol==-1:
                print("Fallo")

1
2
3
4
CPU times: user 809 ms, sys: 6.82 ms, total: 815 ms
Wall time: 817 ms


Comprobar que son minimales

In [90]:
for i in range(1,len(CS1)):
    print(i)
    for j in range(i+1,len(CS1)):
        for caso in CS1[i]:
            for caso2 in CS1[j]:
                if caso.issubset(caso2):
                    print("Fallo")

1
2
3
4
